# **1. Import, load, ...**

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import re
import random
import seaborn as sns
import pickle

from sklearn import metrics
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA


from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/dataset/phase_3_TRAIN_MLPC2021_generic.csv")

In [ ]:
splitted_id_col = data['id'].str.split('-', expand=True) # create 3 columns out of one (the 'id' column)

data2 = data.T[1:] # Remove the first column
data2 = data2.T # Removing the first column done


#data['pianist'] = data3[0]
data2.insert(0, 'pianist',splitted_id_col[0], allow_duplicates=False)
data2.insert(1, 'piece',splitted_id_col[1], allow_duplicates=False)
data2.insert(2, 'snippet',splitted_id_col[2], allow_duplicates=False)

data2

,pianist,piece,snippet,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,essentia_spectral_complexity_mean,essentia_spectral_complexity_stdev,essentia_spectral_rolloff_mean,essentia_spectral_rolloff_stdev,essentia_strong_peak_mean,essentia_strong_peak_stdev,librosa_bpm,librosa_chroma_mean_0,librosa_chroma_var_0,librosa_chroma_pct_10_0,librosa_chroma_pct_50_0,librosa_chroma_pct_90_0,librosa_chroma_mean_1,librosa_chroma_var_1,librosa_chroma_pct_10_1,librosa_chroma_pct_50_1,librosa_chroma_pct_90_1,librosa_chroma_mean_2,librosa_chroma_var_2,librosa_chroma_pct_10_2,librosa_chroma_pct_50_2,librosa_chroma_pct_90_2,librosa_chroma_mean_3,librosa_chroma_var_3,librosa_chroma_pct_10_3,librosa_chroma_pct_50_3,librosa_chroma_pct_90_3,librosa_chroma_mean_4,...,librosa_chroma_var_7,librosa_chroma_pct_10_7,librosa_chroma_pct_50_7,librosa_chroma_pct_90_7,librosa_chroma_mean_8,librosa_chroma_var_8,librosa_chroma_pct_10_8,librosa_chroma_pct_50_8,librosa_chroma_pct_90_8,librosa_chroma_mean_9,librosa_chroma_var_9,librosa_chroma_pct_10_9,librosa_chroma_pct_50_9,librosa_chroma_pct_90_9,librosa_chroma_mean_10,librosa_chroma_var_10,librosa_chroma_pct_10,librosa_chroma_pct_50_10,librosa_chroma_pct_90_10,librosa_chroma_mean_11,librosa_chroma_var_11,librosa_chroma_pct_10_11,librosa_chroma_pct_50_11,librosa_chroma_pct_90_11,librosa_spectral_bandwidth_mean,librosa_spectral_bandwidth_stdev,librosa_spectral_flatness_mean,librosa_spectral_flatness_stdev,midlevel_features_melody,midlevel_features_articulation,midlevel_features_rhythm_complexity,midlevel_features_rhythm_stability,midlevel_features_dissonance,midlevel_features_tonal_stability,midlevel_features_minorness,score_mode,score_key_strength,mean_A,mean_V,quadrant
0,GG,01,000,0.206445,0.131383,13.9988,58.3295,3.1,0.539736,0.193987,3367.79,3684.12,2.98609,2.53088,4423.86,6692.71,0.316206,0.329171,246.094,0.423029,0.146997,0.00885275,0.287094,1,0.339058,0.0895432,0.00749274,0.289652,0.78341,0.372997,0.151724,0.0080173,0.119889,1,0.380705,0.0970007,0.0150791,0.304984,0.855147,0.471226,...,0.158753,0.00542582,0.270609,1,0.295291,0.0876013,0.004258,0.190495,0.764857,0.300751,0.138661,0.00166196,0.0583674,0.975641,0.253962,0.094545,0.00423871,0.0747328,0.765817,0.301682,0.0994945,0.00608847,0.144401,0.815172,3455.58,1781.71,0.139911,0.248388,0.0653973,-0.0293801,-0.303987,-0.0533063,-0.325228,0.347745,-0.119278,0,0.795727,50.6897,2.72414,1
1,GG,01,001,0.145753,0.0579708,3.53326,75.1662,4.2,0.476832,0.123564,1395.53,401.988,4.18192,2.29626,691.307,194.69,0.438665,0.363517,246.094,0.27037,0.112473,0.00850877,0.0879951,0.937823,0.191162,0.0393032,0.00838448,0.107667,0.472149,0.280426,0.145651,0.0111433,0.0751445,1,0.28144,0.0381965,0.0233324,0.270883,0.559445,0.43321,...,0.147123,0.00628117,0.0701699,1,0.141696,0.0263808,0.00505295,0.049705,0.425738,0.12496,0.0592229,0.00424075,0.0409964,0.31199,0.128249,0.0307468,0.0051728,0.0454852,0.455922,0.203408,0.0690659,0.00629769,0.0931326,0.499201,2745.55,721.293,0.000275353,0.000483742,0.35304,-0.0739568,-0.381182,0.12075,-0.479266,0.451572,-0.0642521,0,0.844387,50.6897,2.72414,1
2,GG,01,002,0.149111,0.0590115,3.33737,82.7539,4.3,0.457741,0.132359,1318.76,289.306,4.30359,2.12704,674.839,210.394,0.534696,0.410875,246.094,0.193968,0.0786291,0.00369891,0.0459639,0.661004,0.196718,0.0455848,0.00654415,0.134951,0.538369,0.394663,0.185849,0.00803431,0.111176,1,0.252466,0.0563736,0.0159382,0.169116,0.649735,0.160429,...,0.170728,0.00605498,0.137642,1,0.199484,0.0286001,0.0104906,0.1601,0.427959,0.275843,0.136039,0.00964787,0.0916363,1,0.213787,0.054531,0.00848097,0.0972469,0.541696,0.172331,0.0831675,0.00538214,0.0439865,0.575106,2593.57,583.57,0.000202457,0.000314307,0.201885,0.0169443,-0.337422,0.103853,-0.388214,0.430684,-0.226197,0,0.758119,50.6897,2.72414,1
3,GG,01,003,0.163914,0.0574

# **2. Balancing our data (pls use data_balanced for working in upcoming tasks)**

Samples of class 2 are in 4. doubled. 

# **3. feature selection**

In [ ]:
feature_group_1 = data.iloc[:, 4:19]
feature_group_2 = data.iloc[:, 19:84]
feature_group_3 = data.iloc[:, 84:91]
feature_group_4 = data.iloc[:, 91:-1]
target = data.iloc[:, -1:]

# **4. Use cross-validation for all your experiments. Decide how to split up the data into folds, based on what you know about the data.**

Each fold serves as test set once. Each test set should represent the dataset/reality as adequat as possible.<br>
Pieces: 6 pianists x 47 pieces (read also 48?) x ca. 10 snippets (not fixed!) per piece <br>
<br>
Problem: each piece has a different time span. <br>
Let's look at the distributions, we have to keep balance of the used labels and maybe the pianists. 

Idea: we could create small dataframes each containing a certain pianist and a certain quadrant/class. Then randomly sort the samples/rows to the folds.<br>
(We could also include selection after snippets but I don't think that's important in this case.)

How many folds shall we choose?

Back to the question how many folds to use: A glimpse in the internet showed that often 5, 10 or 20 folds are used and reach a good accuracy. The webpage I saw meant that a good trade-off between speed and accuracy would be 10 folds. So I use it here but we can also change it or experiement with different number of folds. <br>
Be aware that in the following I try to build random sample folds with good representation of the data. Therefore, we would have to make changes in the following box by modifying fold number:

## **Create folds**

In [ ]:
data2['quadrant'].value_counts()

3    1380
4     480
1     473
2     274
Name: quadrant, dtype: int64

In [ ]:

for i in range(7):#10):
    if i == 0:
        name = 'fold_df' + str(i)
        foldname_list = [name]
    else:
        name = 'fold_df'+ str(i)
        foldname_list.append(name)
foldname_list

['fold_df0',
 'fold_df1',
 'fold_df2',
 'fold_df3',
 'fold_df4',
 'fold_df5',
 'fold_df6']

Update: we try out only 7 folds: Reason: With 10 folds fold8 and fold9 only has samples from class 1 and 3. Furthermore, fold7 only would have 8 samples from class 2

In [ ]:
def small_df(pianist_name, data):
    data_onlyOnePian = data[data['pianist']==pianist_name]
    
    data_small1 = data_onlyOnePian[data_onlyOnePian['quadrant']==1]
    data_small2 = data_onlyOnePian[data_onlyOnePian['quadrant']==2]
    data_small3 = data_onlyOnePian[data_onlyOnePian['quadrant']==3]
    data_small4 = data_onlyOnePian[data_onlyOnePian['quadrant']==4]
    
    return data_small1,data_small2,data_small3,data_small4

In [ ]:
random.seed(0) # here enogh or below by random_piece = ?????
print('1. I used in next line the dataset before the feature selection!')
pianists = data2['pianist'].value_counts() # build list with pianist names


for single_pian in pianists.keys():
    print('2. I used in next line the dataset before the feature selection!')
    data_small1, data_small2, data_small3, data_small4  = small_df(single_pian, data2) # with this
    # function we now have small dataframes. Each on theme is unique in Pianist x Quadrant combination. 
    # Per dataframe only one pianist and only one quadrant are given.
    # E. g.: one of data_small1: pianist=RT, quadrant=1
    dataset_list =  [data_small1, data_small2, data_small3, data_small4]

    # Problem solver:
    # When we loop over the foldname_list then fold_df0 would get always a piece but not fold_df6. Therefore, we change the order of folds:
    
    first_ele = foldname_list[0]
    foldname_list = foldname_list[1:]
    foldname_list.append(first_ele)

    # counts only for the first iteration: 
    if single_pian == pianists.keys()[0]: # CREATE the fold dataframes

      for name_fold in foldname_list: # look at each fold we want to create
       
        data_small1_pieces = list(data_small1['piece'])#.count_values()#[data_small1['piece']=='01']
        data_small1_pieces = set(data_small1_pieces)
        
        random_piece = random.sample(data_small1_pieces,1)

        globals()[name_fold] = data_small1[data_small1['piece']==random_piece[0]]

        data_small1 = data_small1[data_small1['piece']!=random_piece[0]] # delete the pieces which we have already used
        
      while data_small1.shape[0] > 0:
        # Choose a random piece:
        data_small1_pieces = list(data_small1['piece'])
        data_small1_pieces = set(data_small1_pieces)
        #while len(single_small_data_pieces) > 0:
        random_piece = random.sample(data_small1_pieces,1)

        # Find fold with the least samples:
        length_list = []
        for i in foldname_list:
          length_single_fold = len(globals()[i])
          length_list.append(length_single_fold)  
        index_min = length_list.index(min(length_list)) # length_list has same order than foldname_list --> we can use index_min for it
        
        # Append fold with the least samples by one piece:
        appender = data_small1[data_small1['piece']==random_piece[0]]
        globals()[foldname_list[index_min]] = pd.concat([globals()[foldname_list[index_min]], appender]) # update the fold_dataframe with new samples

        data_small1 = data_small1[data_small1['piece']!=random_piece[0]] # delete the pieces which we have already used
      # for the upcoming:
      dataset_list = [data_small2, data_small3, data_small4] 

    for single_small_data in dataset_list: # EXPAND fold dataframes

        for name_fold in foldname_list: # look at each fold we want to expand
            single_small_data_pieces = list(single_small_data['piece'])
            single_small_data_pieces = set(single_small_data_pieces)
            if len(single_small_data_pieces) > 0:
              random_piece = random.sample(single_small_data_pieces,1)
              
              appender = single_small_data[single_small_data['piece']==random_piece[0]]

              globals()[name_fold] = pd.concat([globals()[name_fold], appender]) # update the fold_dataframe with new samples

              single_small_data = single_small_data[single_small_data['piece']!=random_piece[0]] # delete the pieces which we have already used

        # When there are more pieces than folds in a data_small... then we have to sort the rest of them to the folds:
        while single_small_data.shape[0] > 0:
          # Choose a random piece:
          single_small_data_pieces = list(single_small_data['piece'])
          single_small_data_pieces = set(single_small_data_pieces)
          #while len(single_small_data_pieces) > 0:
          random_piece = random.sample(single_small_data_pieces,1)

          # Find fold with the least samples:
          length_list = []
          for i in foldname_list:
            length_single_fold = len(globals()[i])
            length_list.append(length_single_fold)  
          index_min = length_list.index(min(length_list)) # length_list has same order than foldname_list --> we can use index_min for it
          
          # Append fold with the least samples by one piece:
          appender = single_small_data[single_small_data['piece']==random_piece[0]]
          globals()[foldname_list[index_min]] = pd.concat([globals()[foldname_list[index_min]], appender]) # update the fold_dataframe with new samples

          single_small_data = single_small_data[single_small_data['piece']!=random_piece[0]] # delete the pieces which we have already used

    
print('\nWe now have 10 fold sets reaching from "fold_df0" up to "fold_df9".')            
fold_df0 # e.g.


1. I used in next line the dataset before the feature selection!
2. I used in next line the dataset before the feature selection!
2. I used in next line the dataset before the feature selection!
2. I used in next line the dataset before the feature selection!
2. I used in next line the dataset before the feature selection!
2. I used in next line the dataset before the feature selection!
2. I used in next line the dataset before the feature selection!

We now have 10 fold sets reaching from "fold_df0" up to "fold_df9".


,pianist,piece,snippet,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,essentia_spectral_complexity_mean,essentia_spectral_complexity_stdev,essentia_spectral_rolloff_mean,essentia_spectral_rolloff_stdev,essentia_strong_peak_mean,essentia_strong_peak_stdev,librosa_bpm,librosa_chroma_mean_0,librosa_chroma_var_0,librosa_chroma_pct_10_0,librosa_chroma_pct_50_0,librosa_chroma_pct_90_0,librosa_chroma_mean_1,librosa_chroma_var_1,librosa_chroma_pct_10_1,librosa_chroma_pct_50_1,librosa_chroma_pct_90_1,librosa_chroma_mean_2,librosa_chroma_var_2,librosa_chroma_pct_10_2,librosa_chroma_pct_50_2,librosa_chroma_pct_90_2,librosa_chroma_mean_3,librosa_chroma_var_3,librosa_chroma_pct_10_3,librosa_chroma_pct_50_3,librosa_chroma_pct_90_3,librosa_chroma_mean_4,...,librosa_chroma_var_7,librosa_chroma_pct_10_7,librosa_chroma_pct_50_7,librosa_chroma_pct_90_7,librosa_chroma_mean_8,librosa_chroma_var_8,librosa_chroma_pct_10_8,librosa_chroma_pct_50_8,librosa_chroma_pct_90_8,librosa_chroma_mean_9,librosa_chroma_var_9,librosa_chroma_pct_10_9,librosa_chroma_pct_50_9,librosa_chroma_pct_90_9,librosa_chroma_mean_10,librosa_chroma_var_10,librosa_chroma_pct_10,librosa_chroma_pct_50_10,librosa_chroma_pct_90_10,librosa_chroma_mean_11,librosa_chroma_var_11,librosa_chroma_pct_10_11,librosa_chroma_pct_50_11,librosa_chroma_pct_90_11,librosa_spectral_bandwidth_mean,librosa_spectral_bandwidth_stdev,librosa_spectral_flatness_mean,librosa_spectral_flatness_stdev,midlevel_features_melody,midlevel_features_articulation,midlevel_features_rhythm_complexity,midlevel_features_rhythm_stability,midlevel_features_dissonance,midlevel_features_tonal_stability,midlevel_features_minorness,score_mode,score_key_strength,mean_A,mean_V,quadrant
2431,RT,29,000,0.238832,0.119022,19.3593,214.198,3.2,0.500805,0.214173,4237.73,4157.26,6.83198,4.70135,1555.93,3268.66,0.707715,0.736735,172.266,0.334186,0.0851746,0.0330018,0.291184,0.890755,0.370433,0.0912717,0.0202551,0.323254,0.943582,0.39371,0.100886,0.0377117,0.340606,1,0.295013,0.0513987,0.0204258,0.287401,0.575312,0.390558,...,0.111358,0.140364,0.54281,1,0.35164,0.0384333,0.0793743,0.368546,0.61084,0.366438,0.127908,0.0158194,0.196386,0.866236,0.374101,0.148691,0.0117226,0.206383,1,0.362549,0.0868926,0.032319,0.31325,0.922635,4640.87,1990.63,0.0157506,0.0463636,-0.125897,0.240984,-0.137616,-0.0876322,-0.264447,0.27186,-0.42986,0,0.813261,96.8966,3.55172,1
2432,RT,29,001,0.180066,0.0629057,1.57398,215.075,3.4,0.605619,0.105316,1711.49,261.31,9.21089,2.37152,708.977,193.203,0.923428,0.482653,172.266,0.190218,0.0376923,0.0342058,0.125045,0.388816,0.436075,0.111016,0.0256423,0.360005,1,0.54821,0.150631,0.0336466,0.48267,1,0.247456,0.0256855,0.0480708,0.24746,0.444564,0.182076,...,0.0915458,0.0127446,0.196392,0.903701,0.335811,0.114263,0.0217852,0.215898,1,0.298326,0.0872646,0.025887,0.173486,0.781907,0.128604,0.0183544,0.0160256,0.0687377,0.363075,0.162362,0.0628935,0.00912868,0.0525712,0.511535,3651.73,419.567,0.000490394,0.000232195,-0.0197328,0.344323,-0.036369,-0.000703514,-0.30002,0.231168,-0.419493,1,0.695062,96.8966,3.55172,1
2433,RT,29,002,0.18494,0.0653144,2.02947,216.717,3.9,0.560036,0.132106,1819.22,308.407,8.79722,2.23568,791.913,210.813,1.02724,0.822674,172.266,0.230438,0.0905375,0.0103977,0.0904549,0.874515,0.175339,0.0483131,0.010267,0.0815465,0.485322,0.202351,0.0452248,0.0146808,0.132789,0.518177,0.408395,0.125602,0.0227991,0.312738,1,0.465946,...,0.0768608,0.032443,0.24123,0.735802,0.294264,0.0946753,0.0126986,0.181796,0.969781,0.308593,0.120287,0.0122195,0.141736,1,0.150719,0.0189337,0.0117734,0.116325,0.342236,0.209161,0.0643681,0.00674182,0.114069,0.570198,3776.22,489.608,0.000581156,0.000408918,-0.0841103,0.348853,-0.0216718,-0.0413387,-0.23486,0.194672,-0.272492,0,0.749013,96.8966,3.55172,1
2434,RT,29,003,0.139036,0.0613731,6.13748

In [ ]:
print('Number per samples per fold:')
index = 0
for i in foldname_list:
  print('fold',index,':')
  index += 1
  print(len(globals()[i]), 'number samples current fold')
  print(globals()[i]['quadrant'].value_counts())

Number per samples per fold:
fold 0 :
366 number samples current fold
3    206
1     65
4     56
2     39
Name: quadrant, dtype: int64
fold 1 :
369 number samples current fold
3    207
4     69
1     65
2     28
Name: quadrant, dtype: int64
fold 2 :
382 number samples current fold
3    197
1     81
4     72
2     32
Name: quadrant, dtype: int64
fold 3 :
391 number samples current fold
3    217
4     65
1     61
2     48
Name: quadrant, dtype: int64
fold 4 :
366 number samples current fold
3    155
1     84
4     81
2     46
Name: quadrant, dtype: int64
fold 5 :
355 number samples current fold
3    186
4     61
2     54
1     54
Name: quadrant, dtype: int64
fold 6 :
378 number samples current fold
3    212
4     76
1     63
2     27
Name: quadrant, dtype: int64


As you can see we roughly keep the distribution of classes per fold. Comes close to the folds of Stratified Cross Validation BUT we keep the WHOLE pieces in the single folds.

In [ ]:
for index, name_fold in enumerate(foldname_list):
    globals()[name_fold]['fold_number'] = index # trick to use a string as a variable name
    
for index, name_fold in enumerate(foldname_list):
    if index == 0:
        data_balanced_complete = globals()[name_fold]
    else:
        data_balanced_complete = pd.concat([data_balanced_complete, globals()[name_fold]])


In [ ]:
# Double the samples of class2 which stay in the same fold
data_class2 = data_balanced_complete[data_balanced_complete.quadrant == 2] # data_reducedThree replaced by data2
# one dataframe in which are only samples from class 2
data_balanced_complete = pd.concat([data_balanced_complete, data_class2]) 

In [ ]:
# Sort the data by folds:
data_balanced_complete = data_balanced_complete.sort_values(by=['fold_number'])

In [ ]:
data_balanced_complete

,pianist,piece,snippet,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,essentia_spectral_complexity_mean,essentia_spectral_complexity_stdev,essentia_spectral_rolloff_mean,essentia_spectral_rolloff_stdev,essentia_strong_peak_mean,essentia_strong_peak_stdev,librosa_bpm,librosa_chroma_mean_0,librosa_chroma_var_0,librosa_chroma_pct_10_0,librosa_chroma_pct_50_0,librosa_chroma_pct_90_0,librosa_chroma_mean_1,librosa_chroma_var_1,librosa_chroma_pct_10_1,librosa_chroma_pct_50_1,librosa_chroma_pct_90_1,librosa_chroma_mean_2,librosa_chroma_var_2,librosa_chroma_pct_10_2,librosa_chroma_pct_50_2,librosa_chroma_pct_90_2,librosa_chroma_mean_3,librosa_chroma_var_3,librosa_chroma_pct_10_3,librosa_chroma_pct_50_3,librosa_chroma_pct_90_3,librosa_chroma_mean_4,...,librosa_chroma_pct_10_7,librosa_chroma_pct_50_7,librosa_chroma_pct_90_7,librosa_chroma_mean_8,librosa_chroma_var_8,librosa_chroma_pct_10_8,librosa_chroma_pct_50_8,librosa_chroma_pct_90_8,librosa_chroma_mean_9,librosa_chroma_var_9,librosa_chroma_pct_10_9,librosa_chroma_pct_50_9,librosa_chroma_pct_90_9,librosa_chroma_mean_10,librosa_chroma_var_10,librosa_chroma_pct_10,librosa_chroma_pct_50_10,librosa_chroma_pct_90_10,librosa_chroma_mean_11,librosa_chroma_var_11,librosa_chroma_pct_10_11,librosa_chroma_pct_50_11,librosa_chroma_pct_90_11,librosa_spectral_bandwidth_mean,librosa_spectral_bandwidth_stdev,librosa_spectral_flatness_mean,librosa_spectral_flatness_stdev,midlevel_features_melody,midlevel_features_articulation,midlevel_features_rhythm_complexity,midlevel_features_rhythm_stability,midlevel_features_dissonance,midlevel_features_tonal_stability,midlevel_features_minorness,score_mode,score_key_strength,mean_A,mean_V,quadrant,fold_number
2204,RT,09,000,0.160764,0.0926286,4.98022,248.999,5.4,0.490525,0.134998,1958.65,1968.15,6.34067,2.39975,1372.58,2732.28,1.42533,1.2307,215.332,0.20349,0.0617915,0.00222012,0.0859268,0.478598,0.34261,0.134694,0.00609832,0.209611,1,0.374753,0.144118,0.00520755,0.254981,1,0.232642,0.0732249,0.00670241,0.0885685,0.676606,0.245973,...,0.00875854,0.0981777,0.634276,0.182162,0.0703988,0.00463748,0.0466499,0.634753,0.239601,0.110574,0.00259893,0.0776225,1,0.177602,0.0543056,0.00188872,0.0550876,0.438817,0.24647,0.137048,0.00165945,0.0413976,1,3249.25,997.285,0.0106028,0.0449987,0.0419518,0.528934,0.0895962,-0.0378641,-0.471597,0.409221,-0.645418,0,0.874966,84.8276,4.03448,1,0
1770,AS,04,004,0.192879,0.0604446,3.00845,123.582,4.5,0.54677,0.117728,862.504,163.233,5.69409,2.45641,659.869,194.898,0.589746,0.366294,172.266,0.233246,0.0908601,0.0142816,0.105575,0.913622,0.228335,0.0461919,0.0127639,0.169102,0.553806,0.343257,0.132693,0.0152644,0.172316,1,0.286005,0.101101,0.0114787,0.166653,1,0.215011,...,0.0121259,0.0958307,1,0.247243,0.0479394,0.0129345,0.191392,0.527401,0.387938,0.152681,0.0180944,0.20496,1,0.277039,0.0764427,0.0196508,0.197561,0.734987,0.18517,0.0451331,0.0184923,0.0996808,0.514644,1004,138.29,5.82651e-06,8.17468e-06,0.0829169,0.314684,0.0233131,0.0156469,-0.311518,0.240168,-0.246771,0,0.677187,53.4483,-0.344828,2,0
1769,AS,04,003,0.176507,0.0669193,3.80249,114.303,5.1,0.51923,0.143566,846.981,193.804,5.10313,2.08393,683.224,188.158,0.704755,0.591174,184.57,0.259966,0.129458,0.00811212,0.0559882,1,0.170007,0.0341236,0.00937783,0.0970137,0.37859,0.286224,0.107805,0.0185339,0.141706,1,0.357462,0.131013,0.02403,0.207992,1,0.20971,...,0.00893122,0.128526,1,0.283386,0.0969427,0.00815225,0.165589,0.901542,0.236641,0.0774504,0.0119295,0.1019,0.622818,0.212947,0.0895406,0.0082719,0.0774832,0.840037,0.178694,0.0467428,0.00686848,0.0650423,0.506164,995.14,131.465,5.10361e-06,5.57037e-06,0.155215,0.308072,-0.0148229,-0.013459,-0.463131,0.357361,-0.616787,1,0.76346,53.4483,-0.344828,2,0
1768,AS,04,002,0.176213,0.0622534,2.78382,113.886,4,0.582187,0.120637,765.744,157.127,

In [ ]:
data_balanced_complete = data_balanced_complete.reset_index() # now each index in data_balanced is unique. This is needed for the
# upcoming part below.
print('Attention: You can run this box only once without error and restarting the notebook.')

Attention: You can run this box only once without error and restarting the notebook.


--> Here we have a new dataframe to work with. It carries all the information about which sample belongs to which fold. Also the 'fold_number' values are sorted. --> Important for Cross validation.

## **X,y for Cross validation**

In [ ]:
#data_balanced_complete = data_balanced_complete.reset_index() # now each index in data_balanced is unique. This is needed for the
# upcoming part below.
#print('Attention: You can run this box only once without error and restarting the notebook.')

In [ ]:
X = data_balanced_complete.drop(['quadrant','fold_number','pianist','snippet','piece','level_0','index'], axis=1)  # drop whatever you want for the WHOLE dataset
print('Dataset X: Here you can drop anything you don\'t need')
X = X.astype('float', errors='ignore')
X = X.to_numpy() # convert dataframe to array
y = data_balanced_complete['quadrant']
y = y.to_numpy(dtype=int)


--> Here we have a new dataframe to work with. It carries all the information about which sample belongs to which fold. Also the 'fold_number' values are sorted. --> Important for Cross validation.

In [ ]:
X = data_balanced_complete.drop(['quadrant','fold_number','pianist','snippet','piece','level_0','index'], axis=1)  # drop whatever you want for the WHOLE dataset
print('Dataset X: Here you can drop anything you don\'t need')
X = X.astype('float', errors='ignore')
X = X.to_numpy() # convert dataframe to array
y = data_balanced_complete['quadrant']
y = y.to_numpy(dtype=int)


Dataset X: Here you can drop anything you don't need


## **Cross Validation with selfmade folds:**

In [ ]:
list_split_indices = []
for fold_index in range(len(foldname_list)):  
    first_index_of_fold = data_balanced_complete[data_balanced_complete['fold_number'] == fold_index].index[0]
    list_split_indices.append(first_index_of_fold)
list_split_indices = list_split_indices[1:] # now we have a list of indices where a new fold starts in a dataframe

In [ ]:
your_estimator = None # put in here

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html
cv_results = cross_validate(your_estimatior, X, y, cv=list_split_indices) # single metric evaluation

In [ ]:
# Alternative:
scores = cross_validate(your_estimator, X, y, cv=list_split_indices, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True) # multimetric evaluation

## **Stratified Cross Validation:**

In [ ]:
your_estimator = None # put in here

In [ ]:
cv_strat = StratifiedKFold(n_splits=10) # you can change n_splits as it works best

In [ ]:

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html
cv_results = cross_validate(your_estimatior, X, y, cv=cv_strat) # single metric evaluation

In [ ]:
# Alternative:
scores = cross_validate(your_estimator, X, y, cv=cv_strat, scoring=('r2', 'neg_mean_squared_error'), return_train_score=True) # multimetric evaluation

# **5. Apply at least four different learning algorithms**

## **Setup:**

In [ ]:
class Selector(BaseEstimator):
    """ A Feature selector for use in the pipeline
    Using this the classifier stores the relevant features itself and we don't have to select them manually if we apply the clf later.
    """
    
    def __init__(self, features):
        super().__init__()
        self.features = features
        
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        return X[self.features]

In [ ]:
def evaluate(clf, X_test, y_test):
    """ simple function to evaluate the classifier
    It plots following: 
    - classifier parameters
    - accuracy
    - classification report
    - confusion matrix
    """
    y_pred = clf.predict(X_test)
    print('model params: \n', clf.get_params)
    print('Accuracy:\n', clf.score(X_test, list(y_test)) )
    print(classification_report(list(y_test), list(y_pred)))
    print('confusion matrix:\n')
    metrics.plot_confusion_matrix(clf, X_test, list(y_test), display_labels=['happy', 'relaxed', 'sad', 'angry'])
    plt.show()